In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/av-janatahr-042020/train.csv
/kaggle/input/av-janatahr-042020/test.csv
/kaggle/input/av-janatahr-042020/sample_submission.csv


In [2]:
#v3: 2400, v23: 673040 #v24 6727

In [3]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import auc,roc_auc_score
from catboost import CatBoostClassifier,Pool, cv
from sklearn.preprocessing import LabelEncoder
from lightgbm import LGBMClassifier
from sklearn.metrics import roc_curve, auc, classification_report, confusion_matrix, accuracy_score
from sklearn.model_selection import train_test_split, cross_val_score, KFold, StratifiedKFold

train = pd.read_csv('../input/av-janatahr-042020/train.csv')
test = pd.read_csv('../input/av-janatahr-042020/test.csv')
sample = pd.read_csv('../input/av-janatahr-042020/sample_submission.csv')
print(train.shape, test.shape, sample.shape)

(18359, 14) (15021, 13) (15021, 2)


In [4]:
df = pd.concat([train, test])
print(df.shape, train.shape, test.shape)

(33380, 14) (18359, 14) (15021, 13)


In [5]:
#2
indx_group = []#

for row in df['city_development_index']:
    if row <= 0.80:
        indx_group.append(0.80)
    elif row >0.80 and  row <= 0.83:
        indx_group.append(0.83)
    elif row > 0.83 and row <= 0.86:
        indx_group.append(0.86)
    elif row > 0.86 and row <= 0.89:
        indx_group.append(0.89)
    elif row > 0.80 and row <= 0.90:
        indx_group.append(0.90)
    elif row > 0.90 and row <= 0.91:
        indx_group.append(0.91)
    elif row > 0.91 and row <= 0.92:
        indx_group.append(0.92)
    elif row > 0.92 and row <= 0.93:
        indx_group.append(0.93)
    elif row > 0.93 and row <= 0.97:
        indx_group.append(0.97)
df['indx_group'] = indx_group
df['indx_group'] = df['indx_group']

In [6]:
#2
#hrs = []
#for row in df['training_hours']:
#    if row <= 5:
#        hrs.append(5)
#    elif row > 5 and row <= 10:
#        hrs.append(10)
#    elif row > 10 and row <= 15:
#        hrs.append(15)
#    elif row > 15 and row <= 20:
#        hrs.append(20)
#    elif row > 20 row <= 25:
#        hrs.append(25)
#    elif row > 25 and row <= 30:
#        hrs.append(30)
#    elif row > 30 and row <= 35:
#        hrs.append(35)
#    elif row > 35 and row <= 40:
#        hrs.append(40)
#    elif row > 40 and row <= 45:
#        hrs.append(45)
#    elif row > 45 and row <= 50:
#        hrs.append(50)
#    elif row > 50 and row <= 60:
#        hrs.append(60)
#    elif row >60 row <= 70:
#        hrs.append(70)
#    elif row >70 and row <= 80:
#        hrs.append(80)
#    elif row >80 and row <= 90:
#        hrs.append(90)
#    elif row >90 and row <= 100:
#        hrs.append(100)
#    elif row <= 130:
#        hrs.append(130)
#    elif row <= 150:
#        hrs.append(150)
#    elif row <= 170:
#        hrs.append(170)
#    elif row <= 200:
#        hrs.append(200)
#    elif row <= 250:
#        hrs.append(250)
#    elif row <= 500:
#        hrs.append(500)
#df['hrs'] = hrs
#df['hrs'] = df['hrs']

In [7]:
df['company_type'].value_counts().sort_index(ascending=False)

Pvt Ltd                18114
Public Sector           1792
Other                    223
NGO                      973
Funded Startup          1880
Early Stage Startup     1029
Name: company_type, dtype: int64

In [8]:
#1
df['company_size'].fillna('99999', inplace=True)
df['company_type'].fillna('unknown', inplace=True)
df['gender'].fillna('Other', inplace=True)
df['enrolled_university'].fillna('unknown', inplace=True)
df['education_level'].fillna('unknown', inplace=True)
df['major_discipline'].fillna('unknown', inplace=True)
df['experience'].fillna('<1', inplace=True)
df['last_new_job'].fillna('never', inplace=True)

In [9]:
#2
comp_size = []

for row in df['company_size']:
    if row == '<10':
        comp_size.append(10)
    elif row == '10/49':
        comp_size.append(50)
    elif row == '50-99':
        comp_size.append(100)
    elif row == '100-500':
        comp_size.append(500)
    elif row == '500-999':
        comp_size.append(1000)
    elif row == '1000-4999':
        comp_size.append(5000)
    elif row == '5000-9999':
        comp_size.append(10000)
    elif row == '10000+':
        comp_size.append(50000)
    else:
        comp_size.append(row)
df['comp_size'] = comp_size
df['comp_size'] = df['comp_size'].astype('int32')

In [10]:
#3
exp = []
for row in df['experience']:
    if row == '<1':
        exp.append('0')
    elif row == '>20':
        exp.append('21')
    else:
        exp.append(row)
df['exp'] = exp
df['exp'] = df['exp'].astype('int32')

In [11]:
#4
lnj = []
for row in df['last_new_job']:
    if row == '>4':
        lnj.append(5)
    elif row == '4':
        lnj.append(4)
    elif row == '3':
        lnj.append(3)
    elif row == '2':
        lnj.append(2)
    elif row == '1':
        lnj.append(1)
    elif row == 'never':
        lnj.append(0)
df['lnj'] = lnj
df['lnj'] = df['lnj'].astype('int32')

In [12]:
#6
df.drop(['company_size', 'experience', 'last_new_job'], axis=1, inplace=True)

In [13]:
#7
cols = ['city', 'city_development_index', 
'gender', 'relevent_experience', 'enrolled_university',
'major_discipline', 'company_type', 'comp_size', 'exp', 'lnj', 'training_hours', 'indx_group',
] #training_hours
for c in cols:
    df['count_'+c] = (df[c].map(df[c].value_counts())).astype('int32')

In [14]:
#7
cols = ['gender', 'relevent_experience', 'enrolled_university',
'major_discipline', 'company_type', 'comp_size', 'lnj', 'training_hours'
] #training_hours
for c in cols:
    df['countexp_'+c] = (df[c].map(df[c].value_counts()))/df['exp']
df['city_indx_div'] = df['city_development_index']/(df['city'].map(df['city'].value_counts()))

In [15]:
#df['uni_edu_maj'] = df['enrolled_university'].astype('str')+'_'+df['education_level'].astype('str')+'_'+df['major_discipline'].astype('str')
#df['type_size'] = df['company_type'].astype('str')+'_'+df['comp_size'].astype('str')
#df['rev_exp'] = df['relevent_experience'].astype('str')+'_'+df['exp'].astype('str')
#df['rev_exp_lnj'] = df['relevent_experience'].astype('str')+'_'+df['exp'].astype('str')+'_'+df['lnj'].astype('str')

In [16]:
#df['cnt_typ_siz_exp'] = df.groupby(['company_type','comp_size'])['exp'].transform('mean')
#df['cnt_typ_lnj_exp'] = df.groupby(['company_type','comp_size', 'lnj'])['exp'].transform('mean') 
##df['cnt_rev_exp_hrs'] = df.groupby(['relevent_experience','exp'])['training_hours'].transform('mean')
##df['cnt_revlnj_exp_hrs'] = df.groupby(['relevent_experience','lnj','exp'])['training_hours'].transform('mean')
#df['cnt_maj_edu_uni_hrs'] = df.groupby(['education_level', 'major_discipline', 'enrolled_university'])['training_hours'].transform('mean')
#df['cnt_maj_edu_uni_exp'] = df.groupby(['education_level', 'major_discipline', 'enrolled_university'])['exp'].transform('mean')
#df['cnt_city_inx_typ'] = df.groupby(['city', 'city_development_index'])['company_type'].transform('count')
#df['cnt_city_typ_siz'] = df.groupby(['city', 'company_type'])['comp_size'].transform('count')
#df['cnt_gndr_edu_maj'] = df.groupby(['gender', 'education_level', 'major_discipline'])['exp'].transform('count')
#df['cnt_maj_edu_exp'] = df.groupby(['education_level', 'major_discipline'])['exp'].transform('count')
#df['cnt_exp_maj_edu_hrs'] = df.groupby(['exp', 'education_level', 'major_discipline'])['training_hours'].transform('count')

In [17]:
#8
df['m_p'] = (np.where(np.logical_or(df['education_level']=='Masters',
                                   df['education_level']=='Phd'), 1, 2)).astype('int32')

In [18]:
#9
grad_m_p = []
for row in df['education_level']:
    if row == 'Masters':
        grad_m_p.append(1)
    elif row == 'Phd':
        grad_m_p.append(1)
    elif row == 'Graduate':
        grad_m_p.append(1)
    else:
        grad_m_p.append(0)
df['grad_m_p'] = grad_m_p
df['grad_m_p'] = df['grad_m_p'].astype('int32')

In [19]:
#10
df['full'] = (np.where(df['enrolled_university']=='Full time course', 1, 2)).astype('int32')
df['stem'] = (np.where(df['major_discipline']=='STEM', 1, 2)).astype('int32')
df['full_stem'] = (np.where(np.logical_and(df['major_discipline']=='STEM',
                                          df['enrolled_university']=='Full time course'), 1, 2)).astype('int32')
df['mp_stem'] = (np.where(np.logical_and(df['major_discipline']=='STEM',
                                         df['m_p']==1), 1, 2)).astype('int32')
df['gradmp_stem'] = (np.where(np.logical_and(df['major_discipline']=='STEM',
                                             df['grad_m_p']==1), 1, 2)).astype('int32')

In [20]:
#11
p_stup = []
for row in df['company_type']:
    if row == 'Pvt Ltd':
        p_stup.append(1)
    elif row == 'Funded Startup':
        p_stup.append(2)
    elif row == 'Early Stage Startup':
        p_stup.append(3)
    else:
        p_stup.append(0)
df['p_stup'] = p_stup

In [21]:
#12
startup = []
for row in df['company_type']:
    if row == 'Funded Startup':
        startup.append(1)
    elif row == 'Early Stage Startup':
        startup.append(2)
    else:
        startup.append(0)
df['startup'] = startup

In [22]:
#13
exp_01 = []
for row in df['exp']:
    if row == 0:
        exp_01.append(1)
    elif row == 1:
        exp_01.append(1)
    else:
        exp_01.append(0)
df['exp_01'] = exp_01
df['exp_01'] = df['exp_01'].astype('int32')

In [23]:
#14
exp_0123 = []
for row in df['exp']:
    if row == 0:
        exp_0123.append(1)
    elif row == 1:
        exp_0123.append(1)
    elif row == 2:
        exp_0123.append(1)
    elif row == 3:
        exp_0123.append(1)
    else:
        exp_0123.append(0)
df['exp_0123'] = exp_0123
df['exp_0123'] = df['exp_0123'].astype('int32')

In [24]:
#15
exp_012345 = []
for row in df['exp']:
    if row == 0:
        exp_012345.append(1)
    elif row == 1:
        exp_012345.append(1)
    elif row == 2:
        exp_012345.append(1)
    elif row == 3:
        exp_012345.append(1)
    elif row == 4:
        exp_012345.append(1)
    elif row == 5:
        exp_012345.append(1)
    else:
        exp_012345.append(0)
df['exp_012345'] = exp_012345
df['exp_012345'] = df['exp_012345'].astype('int32')

In [25]:
#16
exp_0123a = []
for row in df['exp']:
    if row == 0:
        exp_0123a.append(0)
    elif row == 1:
        exp_0123a.append(1)
    elif row == 2:
        exp_0123a.append(2)
    elif row == 3:
        exp_0123a.append(3)
    else:
        exp_0123a.append(10)
df['exp_0123a'] = exp_0123a
df['exp_0123a'] = df['exp_0123a'].astype('int32')

In [26]:
#17
exp_012345a = []
for row in df['exp']:
    if row == 0:
        exp_012345a.append(0)
    elif row == 1:
        exp_012345a.append(1)
    elif row == 2:
        exp_012345a.append(2)
    elif row == 3:
        exp_012345a.append(3)
    elif row == 4:
        exp_012345a.append(4)
    elif row == 5:
        exp_012345a.append(5)
    else:
        exp_012345a.append(10)
df['exp_012345a'] = exp_012345a
df['exp_012345a'] = df['exp_012345a'].astype('int32')

In [27]:
#18
exp_by3 = []
for row in df['exp']:
    if row == 0:
        exp_by3.append(0)
    elif row == 1:
        exp_by3.append(1)
    elif row == 2:
        exp_by3.append(2)
    elif row == 3:
        exp_by3.append(3)
    elif row == 4:
        exp_by3.append(6)
    elif row == 5:
        exp_by3.append(6)
    elif row == 6:
        exp_by3.append(6)
    elif row == 7:
        exp_by3.append(9)
    elif row == 8:
        exp_by3.append(9)
    elif row == 9:
        exp_by3.append(9)
    elif row == 10:
        exp_by3.append(12)
    elif row == 11:
        exp_by3.append(12)
    elif row == 12:
        exp_by3.append(12)
    elif row == 13:
        exp_by3.append(15)
    elif row == 14:
        exp_by3.append(15)
    elif row == 15:
        exp_by3.append(15)
    elif row == 16:
        exp_by3.append(18)
    elif row == 17:
        exp_by3.append(18)    
    elif row == 18:
        exp_by3.append(18)
    elif row == 19:
        exp_by3.append(21)
    elif row == 20:
        exp_by3.append(21)
    elif row == 21:
        exp_by3.append(24)
df['exp_by3'] = exp_by3
df['exp_by3'] = df['exp_by3'].astype('int32')

In [28]:
#19
exp_by5 = []
for row in df['exp']:
    if row == 0:
        exp_by5.append(0)
    elif row == 1:
        exp_by5.append(1)
    elif row == 2:
        exp_by5.append(2)
    elif row == 3:
        exp_by5.append(3)
    elif row == 4:
        exp_by5.append(4)
    elif row == 5:
        exp_by5.append(5)
    elif row == 6:
        exp_by5.append(10)
    elif row == 7:
        exp_by5.append(10)
    elif row == 8:
        exp_by5.append(10)
    elif row == 9:
        exp_by5.append(10)
    elif row == 10:
        exp_by5.append(10)
    elif row == 11:
        exp_by5.append(15)
    elif row == 12:
        exp_by5.append(15)
    elif row == 13:
        exp_by5.append(15)
    elif row == 14:
        exp_by5.append(15)
    elif row == 15:
        exp_by5.append(15)
    elif row == 16:
        exp_by5.append(20)
    elif row == 17:
        exp_by5.append(20)    
    elif row == 18:
        exp_by5.append(20)
    elif row == 19:
        exp_by5.append(20)
    elif row == 20:
        exp_by5.append(20)
    elif row == 21:
        exp_by5.append(25)
df['exp_by5'] = exp_by5
df['exp_by5'] = df['exp_by5'].astype('int32')

In [29]:
#20
cols = ['exp_01', 'exp_0123', 'exp_012345', 'exp_0123a', 'exp_012345a', 'exp_by3', 'exp_by5'] #training_hours
for c in cols:
    df['count_'+c] = (df[c].map(df[c].value_counts())).astype('int32')

In [30]:
#21
lj_12 = []
for row in df['lnj']:
    if row == 0:
        lj_12.append(0)
    elif row == 1:
        lj_12.append(1)
    elif row == 2:
        lj_12.append(2)
    elif row == 3:
        lj_12.append(0)
    elif row == 4:
        lj_12.append(0)
    elif row == 5:
        lj_12.append(0)
df['lj_12'] = lj_12
df['lj_12'] = df['lj_12'].astype('int32')

In [31]:
#22
lj_123 = []
for row in df['lnj']:
    if row == 0:
        lj_123.append(0)
    elif row == 1:
        lj_123.append(1)
    elif row == 2:
        lj_123.append(2)
    elif row == 3:
        lj_123.append(3)
    elif row == 4:
        lj_123.append(0)
    elif row == 5:
        lj_123.append(0)
df['lj_123'] = lj_123
df['lj_123'] = df['lj_123'].astype('int32')

In [32]:
#23
lj_12a = []
for row in df['lnj']:
    if row == 0:
        lj_12a.append(0)
    elif row == 1:
        lj_12a.append(1)
    elif row == 2:
        lj_12a.append(1)
    elif row == 3:
        lj_12a.append(0)
    elif row == 4:
        lj_12a.append(0)
    elif row == 5:
        lj_12a.append(0)
df['lj_12a'] = lj_12a
df['lj_12a'] = df['lj_12a'].astype('int32')

In [33]:
#24
cs_1050 = []
for row in df['comp_size']:
    if row == 10:
        cs_1050.append(1)
    elif row == 50:
        cs_1050.append(1)
    else:
        cs_1050.append(0)
df['cs_1050'] = cs_1050
df['cs_1050'] = df['cs_1050'].astype('int32')

In [34]:
#25
cs_1050100 = []
for row in df['comp_size']:
    if row == 10:
        cs_1050100.append(1)
    elif row == 50:
        cs_1050100.append(1)
    elif row == 100:
        cs_1050100.append(1)
    else:
        cs_1050100.append(0)
df['cs_1050100'] = cs_1050100
df['cs_1050100'] = df['cs_1050100'].astype('int32')

In [35]:
#26
cs_10500 = []
for row in df['comp_size']:
    if row == 10:
        cs_10500.append(1)
    elif row == 50:
        cs_10500.append(1)
    elif row == 100:
        cs_10500.append(1)
    elif row == 500:
        cs_10500.append(1)
    else:
        cs_10500.append(0)
df['cs_10500'] = cs_10500
df['cs_10500'] = df['cs_10500'].astype('int32')

In [36]:
#27
cs_1000 = []
for row in df['comp_size']:
    if row == 10:
        cs_1000.append(10)
    elif row == 50:
        cs_1000.append(50)
    elif row == 100:
        cs_1000.append(100)
    elif row == 500:
        cs_1000.append(500)
    elif row == 1000:
        cs_1000.append(1000)
    else:
        cs_1000.append(10000)
df['cs_1000'] = cs_1000
df['cs_1000'] = df['cs_1000'].astype('int32')

In [37]:
#28
def city_most(columnname, limit = 500):
    vc = df[columnname].value_counts()
    common = vc > limit
    common = set(common.index[common].values)
    df.loc[df[columnname].map(lambda x: x not in common), columnname] = 'city_9999'
    print("now there are", df[columnname].nunique(), "categories in df")

In [38]:
#29
df['city_copy'] = df['city']

In [39]:
#30
city_most('city_copy')

now there are 14 categories in df


In [40]:
#31
city_group = []
for row in df['city']:
    if row == 'city_103':
        city_group.append(1)
    elif row == 'city_21':
        city_group.append(1)
    elif row == 'city_16':
        city_group.append(1)
    elif row == 'city_114':
        city_group.append(1)
    elif row == 'city_160':
        city_group.append(1)
    elif row == 'city_136':
        city_group.append(1)
    elif row == 'city_67':
        city_group.append(1)
    elif row == 'city_75':
        city_group.append(1)
    elif row == 'city_102':
        city_group.append(1)
    elif row == 'city_104':
        city_group.append(1)
    elif row == 'city_71':
        city_group.append(1)
    elif row == 'city_73':
        city_group.append(1)
    elif row == 'city_100':
        city_group.append(1)
        
    elif row == 'city_61':
        city_group.append(2)
    elif row == 'city_28':
        city_group.append(2)
    elif row == 'city_23':
        city_group.append(2)
    elif row == 'city_65':
        city_group.append(2)
    elif row == 'city_90':
        city_group.append(2)
    elif row == 'city_36':
        city_group.append(2)
        
    elif row == 'city_173':
        city_group.append(3)
    elif row == 'city_11':
        city_group.append(3)
    elif row == 'city_50':
        city_group.append(3)
    elif row == 'city_83':
        city_group.append(3)
    elif row == 'city_116':
        city_group.append(3)
    elif row == 'city_64':
        city_group.append(3)
    elif row == 'city_46':
        city_group.append(3)
    elif row == 'city_162':
        city_group.append(3)
    elif row == 'city_45':
        city_group.append(3)
    elif row == 'city_138':
        city_group.append(3)
    elif row == 'city_97':
        city_group.append(3)#

    elif row == 'city_149':
        city_group.append(4)
    elif row == 'city_57':
        city_group.append(4)
    elif row == 'city_10':
        city_group.append(4)
    elif row == 'city_19':
        city_group.append(4)
    elif row == 'city_159':
        city_group.append(4)
    elif row == 'city_98':
        city_group.append(4)
    elif row == 'city_105':
        city_group.append(4)
    elif row == 'city_99':
        city_group.append(4)
    elif row == 'city_165':
        city_group.append(4)
    elif row == 'city_41':
        city_group.append(4)
    elif row == 'city_123':
        city_group.append(4)
    elif row == 'city_40':
        city_group.append(4)
    elif row == 'city_150':
        city_group.append(4)
    elif row == 'city_89':
        city_group.append(4)
    elif row == 'city_74':
        city_group.append(4)
    elif row == 'city_27':
        city_group.append(4)
    elif row == 'city_24':
        city_group.append(4)
    elif row == 'city_115':
        city_group.append(4)
    elif row == 'city_128':
        city_group.append(4)
    elif row == 'city_13':
        city_group.append(4)#

    elif row == 'city_158':
        city_group.append(5)
    elif row == 'city_142':
        city_group.append(5)
    elif row == 'city_152':
        city_group.append(5)
    elif row == 'city_101':
        city_group.append(5)
    elif row == 'city_91':
        city_group.append(5)
    elif row == 'city_134':
        city_group.append(5)
    elif row == 'city_76':
        city_group.append(5)
    elif row == 'city_70':
        city_group.append(5)
    elif row == 'city_143':
        city_group.append(5)
    elif row == 'city_145':
        city_group.append(5)
    elif row == 'city_1':
        city_group.append(5)
    elif row == 'city_77':
        city_group.append(5)
    elif row == 'city_144':
        city_group.append(5)
    elif row == 'city_20':
        city_group.append(5)
    elif row == 'city_93':
        city_group.append(5)
    elif row == 'city_14':
        city_group.append(5)
    
    else:
        city_group.append(6)#

df['city_group'] = city_group
df['city_group'] = df['city_group'].astype('int32')

In [41]:
#32
#mp
df['mpst_01_lj12'] = df['mp_stem'].astype('str')+'_'+df['exp_01'].astype('str')+'_'+df['lj_12'].astype('str')
df['mpst_0123_lj12'] = df['mp_stem'].astype('str')+'_'+df['exp_0123'].astype('str')+'_'+df['lj_12'].astype('str')
df['mpst_012345_lj12'] = df['mp_stem'].astype('str')+'_'+df['exp_012345'].astype('str')+'_'+df['lj_12'].astype('str')
df['mpst_0123a_lj12'] = df['mp_stem'].astype('str')+'_'+df['exp_0123a'].astype('str')+'_'+df['lj_12'].astype('str')
df['mpst_012345a_lj12'] = df['mp_stem'].astype('str')+'_'+df['exp_012345a'].astype('str')+'_'+df['lj_12'].astype('str')
#grad_mp
df['gmpst_01_lj12'] = df['gradmp_stem'].astype('str')+'_'+df['exp_01'].astype('str')+'_'+df['lj_12'].astype('str')
df['gmpst_0123_lj12'] = df['gradmp_stem'].astype('str')+'_'+df['exp_0123'].astype('str')+'_'+df['lj_12'].astype('str')
df['gmpst_012345_lj12'] = df['gradmp_stem'].astype('str')+'_'+df['exp_012345'].astype('str')+'_'+df['lj_12'].astype('str')
df['gmpst_0123a_lj12'] = df['gradmp_stem'].astype('str')+'_'+df['exp_0123a'].astype('str')+'_'+df['lj_12'].astype('str')
df['gmpst_012345a_lj12'] = df['gradmp_stem'].astype('str')+'_'+df['exp_012345a'].astype('str')+'_'+df['lj_12'].astype('str')
#mp_rev
df['mpstrev_01_lj12'] = df['mp_stem'].astype('str')+'_'+df['exp_01'].astype('str')+'_'+df['relevent_experience'].astype('str')+'_'+df['lj_12'].astype('str')
df['mpstrev_0123_lj12'] = df['mp_stem'].astype('str')+'_'+df['exp_0123'].astype('str')+'_'+df['relevent_experience'].astype('str')+'_'+df['lj_12'].astype('str')
df['mpstrev_012345_lj12'] = df['mp_stem'].astype('str')+'_'+df['exp_012345'].astype('str')+'_'+df['relevent_experience'].astype('str')+'_'+df['lj_12'].astype('str')
df['mpstrev_0123a_lj12'] = df['mp_stem'].astype('str')+'_'+df['exp_0123a'].astype('str')+'_'+df['relevent_experience'].astype('str')+'_'+df['lj_12'].astype('str')
df['mpstrev_012345a_lj12'] = df['mp_stem'].astype('str')+'_'+df['exp_012345a'].astype('str')+'_'+df['relevent_experience'].astype('str')+'_'+df['lj_12'].astype('str')
#grad_rev
df['gmpstrev_01_lj12'] = df['gradmp_stem'].astype('str')+'_'+df['exp_01'].astype('str')+'_'+df['relevent_experience'].astype('str')+'_'+df['lj_12'].astype('str')
df['gmpstrev_0123_lj12'] = df['gradmp_stem'].astype('str')+'_'+df['exp_0123'].astype('str')+'_'+df['relevent_experience'].astype('str')+'_'+df['lj_12'].astype('str')
df['gmpstrev_012345_lj12'] = df['gradmp_stem'].astype('str')+'_'+df['exp_012345'].astype('str')+'_'+df['relevent_experience'].astype('str')+'_'+df['lj_12'].astype('str')
df['gmpstrev_0123a_lj12'] = df['gradmp_stem'].astype('str')+'_'+df['exp_0123a'].astype('str')+'_'+df['relevent_experience'].astype('str')+'_'+df['lj_12'].astype('str')
df['gmpstrev_012345a_lj12'] = df['gradmp_stem'].astype('str')+'_'+df['exp_012345a'].astype('str')+'_'+df['relevent_experience'].astype('str')+'_'+df['lj_12'].astype('str')
#startup_1050, 1050100
df['stup_1050'] = df['startup'].astype('str')+'_'+df['cs_1050'].astype('str')+'_'+df['city'].astype('str')
df['stup_1050100'] = df['startup'].astype('str')+'_'+df['cs_1050100'].astype('str')+'_'+df['city'].astype('str')
#df['stup_1050_a'] = df['exp_0123'].astype('str')+'_'+df['startup'].astype('str')+'_'+df['cs_1050'].astype('str')
#df['stup_1050100_a'] = df['exp_0123'].astype('str')+'_'+df['startup'].astype('str')+'_'+df['cs_1050100'].astype('str')
#df['stup_1050_b'] = df['exp_0123a'].astype('str')+'_'+df['startup'].astype('str')+'_'+df['cs_1050'].astype('str')
#df['stup_1050100_b'] = df['exp_0123a'].astype('str')+'_'+df['startup'].astype('str')+'_'+df['cs_1050100'].astype('str')
df['pstup_500'] = df['p_stup'].astype('str')+'_'+df['cs_10500'].astype('str')+'_'+df['city'].astype('str')
df['pstup_1000'] = df['p_stup'].astype('str')+'_'+df['cs_1000'].astype('str')+'_'+df['city'].astype('str')

In [42]:
#33
cols = ['mpst_01_lj12', 'mpst_0123_lj12', 'mpst_012345_lj12', 'mpst_0123a_lj12', 'mpst_012345a_lj12',
'gmpst_01_lj12', 'gmpst_0123_lj12', 'gmpst_012345_lj12', 'gmpst_0123a_lj12', 'gmpst_012345a_lj12',
'mpstrev_01_lj12', 'mpstrev_0123_lj12', 'mpstrev_012345_lj12', 'mpstrev_0123a_lj12', 'mpstrev_012345a_lj12',
'gmpstrev_01_lj12', 'gmpstrev_0123_lj12', 'gmpstrev_012345_lj12', 'gmpstrev_0123a_lj12', 'gmpstrev_012345a_lj12',
'stup_1050', 'stup_1050100', 'pstup_500', 'pstup_1000', 'indx_group',
#'lj_12','lj_123',
#'exp_by3', 'exp_by5',
#'exp_01', 'exp_0123',
    ]
for c in cols:
    df['count_'+c] = (df[c].map(df[c].value_counts())).astype('int32')

In [43]:
#33a
cat_cols = ['city', 'gender', 'relevent_experience', 'enrolled_university', 'education_level',
'major_discipline', 'company_type', 'comp_size', 'exp', 'lnj', 
'exp_01', 'exp_0123', 'exp_012345', 'exp_0123a', 'exp_012345a',
 'stup_1050', 'stup_1050100', 
'm_p', 'grad_m_p', 'p_stup', 'startup', 'cs_1050', 'cs_1050100', 'cs_10500', 'cs_1000',
#'city_group', 'city_copy',
'mpst_01_lj12', 'mpst_0123_lj12', 'mpst_0123a_lj12',
'gmpst_01_lj12', 'gmpst_0123_lj12', 'gmpst_0123a_lj12',
'mpstrev_01_lj12', 'mpstrev_0123_lj12', 'mpstrev_0123a_lj12',
'gmpstrev_01_lj12', 'gm[pstrev_0123_lj12', 'gmpstrev_0123a_lj12',
]

In [44]:
#34
cols = ['count_mpstrev_01_lj12', 'count_mpstrev_0123_lj12', 'count_mpstrev_012345_lj12', 'count_mpstrev_0123a_lj12', 'count_mpstrev_012345a_lj12',
'count_gmpstrev_01_lj12', 'count_gmpstrev_0123_lj12', 'count_gmpstrev_012345_lj12', 'count_gmpstrev_0123a_lj12', 'count_gmpstrev_012345a_lj12']
for c in cols:
    df['divlj12_'+c] = df[c]/df['lj_12'].map(df['lj_12'].value_counts())
    #df['divlj123_'+c] = df[c]/df['lj_123'].map(df['lj_123'].value_counts())
    #df['divst50_'+c] = df[c]/df['city_group'].map(df['city_group'].value_counts())
    #df['divst100_'+c] = df[c]/df['exp'].map(df['exp'].value_counts())

In [45]:
#35
cols = ['count_mpstrev_01_lj12', 'count_mpstrev_0123_lj12', 'count_mpstrev_012345_lj12', 'count_mpstrev_0123a_lj12', 'count_mpstrev_012345a_lj12']
for c in cols:
    df['divmp_'+c] = df[c]/df['m_p'].map(df['m_p'].value_counts())
    #df['divmp1_'+c] = df[c]/df['full_stem'].map(df['full_stem'].value_counts())
cols = ['count_gmpstrev_01_lj12', 'count_gmpstrev_0123_lj12', 'count_gmpstrev_012345_lj12', 'count_gmpstrev_0123a_lj12', 'count_gmpstrev_012345a_lj12']
for c in cols:
    df['divgmp_'+c] = df[c]/df['grad_m_p'].map(df['grad_m_p'].value_counts())
    #df['divgmp1_'+c] = df[c]/df['full_stem'].map(df['full_stem'].value_counts())

In [46]:
#36
df['div_01'] = df['count_mpstrev_01_lj12']/df['exp_01'].map(df['exp_01'].value_counts())
df['div_0123'] = df['count_mpstrev_0123_lj12']/df['exp_0123'].map(df['exp_0123'].value_counts())
df['div_012345'] = df['count_mpstrev_012345_lj12']/df['exp_012345'].map(df['exp_012345'].value_counts())
df['div_0123a'] = df['count_mpstrev_0123a_lj12']/df['exp_0123a'].map(df['exp_0123a'].value_counts())
df['div_012345a'] = df['count_mpstrev_012345a_lj12']/df['exp_012345a'].map(df['exp_012345a'].value_counts())

df['gdiv_01'] = df['count_gmpstrev_01_lj12']/df['exp_01'].map(df['exp_01'].value_counts())
df['gdiv_0123'] = df['count_gmpstrev_0123_lj12']/df['exp_0123'].map(df['exp_0123'].value_counts())
df['gdiv_012345'] = df['count_gmpstrev_012345_lj12']/df['exp_012345'].map(df['exp_012345'].value_counts())
df['gdiv_0123a'] = df['count_gmpstrev_0123a_lj12']/df['exp_0123a'].map(df['exp_0123a'].value_counts())
df['gdiv_012345a'] = df['count_gmpstrev_012345a_lj12']/df['exp_012345a'].map(df['exp_012345a'].value_counts())

In [47]:
#df['sdiv_01'] = df['count_stup_1050']/df['city'].map(df['exp_01'].value_counts())
#df['sdiv_0123'] = df['count_stup_1050100']/df['exp_01'].map(df['exp_0123'].value_counts())
#df['sdiv_012345'] = df['count_pstup_1050']/df['exp_0123'].map(df['exp_012345'].value_counts())
#df['sdiv_0123a'] = df['count_pstup_1050100']/df['exp_0123'].map(df['exp_0123a'].value_counts())
#df['sdiv_012345a'] = df['count_stup_1050_b']/df['exp_0123a'].map(df['exp_012345a'].value_counts())
#df['sdiv_012345a'] = df['count_stup_1050100_b']/df['exp_0123a'].map(df['exp_012345a'].value_counts())

In [48]:
#37
cols = ['mpstrev_012345a_lj12', 'gmpstrev_012345a_lj12', #'pstup_1000'
       ]
for c in cols:
    df['minexp_'+c] = df.groupby([c])['exp'].transform('min')
    df['meanexp_'+c] = df.groupby([c])['exp'].transform('mean')
    df['minhrs_'+c] = df.groupby([c])['training_hours'].transform('min')
    df['meanhrs_'+c] = df.groupby([c])['training_hours'].transform('mean')
    df['maxindx_'+c] = df.groupby([c])['city_development_index'].transform('max')
    df['meanindx_'+c] = df.groupby([c])['city_development_index'].transform('mean')
    
df['ct1_'] = df.groupby(['gmpstrev_012345a_lj12', 'stup_1050100'])['exp'].transform('count')
#df['ct1cc_'] = df.groupby(['gmpstrev_012345a_lj12', 'stup_1050100'])['exp'].cumcount()
df['min1_'] = df.groupby(['gmpstrev_012345a_lj12', 'stup_1050100'])['exp'].transform('min')
df['mean1_'] = df.groupby(['gmpstrev_012345a_lj12', 'stup_1050100'])['exp'].transform('mean')
 
df['ct2_'] = df.groupby(['gmpstrev_012345a_lj12', 'exp'])['training_hours'].transform('count')
#df['ct2cc_'] = df.groupby(['gmpstrev_012345a_lj12', 'exp'])['training_hours'].cumcount()
df['min2_'] = df.groupby(['gmpstrev_012345a_lj12', 'exp'])['training_hours'].transform('min')
df['mean2_'] = df.groupby(['gmpstrev_012345a_lj12', 'exp'])['training_hours'].transform('mean')

#df['ct3_'] = df.groupby(['city', 'exp'])['city_development_index'].transform('count')
#df['min3_'] = df.groupby(['city', 'exp'])['city_development_index'].transform('max')
#df['mean3_'] = df.groupby(['city', 'exp'])['city_development_index'].transform('mean')

#df['ct3_'] = df.groupby(['city_group', 'exp','gmpstrev_012345a_lj12'])['city_development_index'].transform('count')
#df['min3_'] = df.groupby(['city_group', 'exp','gmpstrev_012345a_lj12'])['city_development_index'].transform('min')
#df['mean3_'] = df.groupby(['city_group', 'exp','gmpstrev_012345a_lj12'])['city_development_index'].transform('mean') 

In [49]:
#df['mean_2_1'] = df['mean2_']/df['mean1_']
#df['min_2_1'] = df['min2_']/df['min1_']
#df['count_2_1'] = df['ct2_']/df['ct1_']

In [50]:
#df['exp_mean_1'] = df['exp']-(df.groupby(['gmpstrev_012345a_lj12', 'stup_1050100'])['exp'].transform('min'))
#df['exp_mean_2'] = df['exp']-(df.groupby(['gmpst_012345a_lj12', 'stup_1050100'])['exp'].transform('min'))
#df['exp_mean_3'] = df['exp']-(df.groupby(['mpstrev_012345a_lj12', 'stup_1050100'])['exp'].transform('mean'))
#df['exp_mean_4'] = df['exp']-(df.groupby(['mpst_012345a_lj12', 'stup_1050100'])['exp'].transform('mean'))

#df['hrs_mean_1'] = df['training_hours']-(df.groupby(['gmpstrev_012345a_lj12', 'exp'])['training_hours'].transform('mean'))
#df['hrs_mean_2'] = df['training_hours']-(df.groupby(['gmpst_012345a_lj12', 'exp'])['training_hours'].transform('mean'))
#df['hrs_mean_3'] = df['training_hours']-(df.groupby(['mpstrev_012345a_lj12', 'exp'])['training_hours'].transform('mean'))
#df['hrs_mean_4'] = df['training_hours']-(df.groupby(['mpst_012345a_lj12', 'exp'])['training_hours'].transform('mean'))

In [51]:
df['exp_hrs_mean'] = df.groupby(['exp'])['training_hours'].transform('mean')
df['indx_hrs_mean'] = df.groupby(['city_development_index'])['training_hours'].transform('mean')
#df['indx_exp'] = df['indx_hrs_mean']/df['exp_hrs_mean']

In [52]:
#38
cols = [c for c in df.columns if df[c].dtypes=='object']
for c in cols:
    le = LabelEncoder()
    df[c] = (le.fit_transform(df[c])).astype('int64')

In [53]:
#cols = ['gender', 'relevent_experience', 'enrolled_university', 'education_level',
#'major_discipline', 'company_type', 'comp_size', 'exp', 'lnj', 
#'exp_01', 'exp_0123', 'exp_012345', 'exp_0123a', 'exp_012345a',
#'m_p', 'grad_m_p', 'p_stup', 'startup', 'cs_1050', 'cs_1050100',# 'cs_10500', 'cs_1000',
#'city_group', 'city_copy'
#       ]
#for c in cols:
#    df[c] = df[c].astype('category')

In [54]:
#39
df_train=df[df['target'].isnull()==False].copy()
df_test=df[df['target'].isnull()==True].copy()

print(df_train.shape,df_test.shape)

(18359, 166) (15021, 166)


In [55]:
df_train['target'] = df_train['target'].astype('int32')

In [56]:
#40
col_to_drop=['enrollee_id','target']
X,y=df_train.drop(col_to_drop,axis=1),df_train['target']
Xtest=df_test.drop(col_to_drop,axis=1)

In [57]:
#41
#print(X.shape,Xtest.shape)
#X_train,X_val,y_train,y_val = train_test_split(X,y,test_size=0.3,random_state = 1994,stratify=y)

In [58]:
err=[]
y_pred_tot=[]

feature_importance_df = pd.DataFrame()

from sklearn.model_selection import KFold,StratifiedKFold
fold=StratifiedKFold(n_splits=5,shuffle=True,random_state=1994)
i=1
for train_index, test_index in fold.split(X,y):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y[train_index], y[test_index]
    m=LGBMClassifier(n_estimators=5000,
                     random_state=1994,
                     learning_rate=0.01,
                     num_leaves=31,
                     max_depth=-1,
                     bagging_frequency=4,
                     bagging_fraction=0.3,
                     bagging_seed=1994,
                     min_data_in_leaf=40,
                     feature_fraction=0.2
                     #reg_alpha=0.2,colsample_bytree=0.5
                    )
    m.fit(X_train,y_train,eval_set=[(X_train,y_train),(X_test, y_test)],eval_metric='auc',
          early_stopping_rounds=500,verbose=100)
    
    preds=m.predict_proba(X_test,num_iteration=m.best_iteration_)[:,-1]
    
    fold_importance_df = pd.DataFrame()
    fold_importance_df["feature"] = X_train.columns
    fold_importance_df["importance"] = m.feature_importances_
    fold_importance_df["fold"] = i + 1
    feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)
    
    
    print("err: ",roc_auc_score(y_test,preds))
    err.append(roc_auc_score(y_test,preds))
    p = m.predict_proba(Xtest, num_iteration=m.best_iteration_)[:,-1]
    i=i+1
    y_pred_tot.append(p)

Training until validation scores don't improve for 500 rounds
[100]	training's auc: 0.748293	training's binary_logloss: 0.357292	valid_1's auc: 0.648575	valid_1's binary_logloss: 0.376296
[200]	training's auc: 0.777399	training's binary_logloss: 0.342263	valid_1's auc: 0.652014	valid_1's binary_logloss: 0.374531
[300]	training's auc: 0.803786	training's binary_logloss: 0.331055	valid_1's auc: 0.651085	valid_1's binary_logloss: 0.374758
[400]	training's auc: 0.826244	training's binary_logloss: 0.32156	valid_1's auc: 0.648569	valid_1's binary_logloss: 0.375714
[500]	training's auc: 0.84404	training's binary_logloss: 0.313374	valid_1's auc: 0.646841	valid_1's binary_logloss: 0.376614
[600]	training's auc: 0.859051	training's binary_logloss: 0.305972	valid_1's auc: 0.644274	valid_1's binary_logloss: 0.377723
Early stopping, best iteration is:
[197]	training's auc: 0.776587	training's binary_logloss: 0.342636	valid_1's auc: 0.652348	valid_1's binary_logloss: 0.374475
err:  0.652347972918331

In [59]:
np.mean(err,0) #0.6724081054054132 #0.6725543976933509 #0.6727011097324567

0.6736387148066815

In [60]:
sample['target']=np.mean(y_pred_tot,0)
sample.head()

,enrollee_id,target
0,16548,0.348758
1,12036,0.088692
2,11061,0.239933
3,5032,0.082245
4,17599,0.138697


In [61]:
sum(sample.target>=0.5)

0

In [62]:
sample.to_csv('sub_673638714lgbm_indxonly.csv',index=False)
sample.shape

(15021, 2)